<a href="https://colab.research.google.com/github/bye23mj/BERT/blob/main/codeBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import sys
print(sys.version)

3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]


In [33]:
import tensorflow as tf
import os

tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.12.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

In [ ]:
#!pip install torch==1.12.1
!pip install transformers==4.23.1
!pip install filelock

In [ ]:
!pip install gdown

In [4]:
!pip install tree_sitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.20.1-cp37-cp37m-linux_x86_64.whl size=329980 sha256=00297bb4c44d485de5a548e7ef3bcb9f6a2213d83a90ce2d22d9f3245247a398
  Stored in directory: /root/.cache/pip/wheels/fa/a1/61/ec5131dac9344581d9e0364bc231721138ebd1318fba66982c
Successfully built tree-sitter


In [26]:
#mkdir data data/code2nl

In [27]:
#mkdir data model/java

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘model/java’: File exists


In [4]:
#cd data/code2nl

/content/data/code2nl


In [6]:
#!gdown https://drive.google.com/uc?id=1rd2Tc6oUWBo7JouwexW3ksQ0PaOhUr6h

Downloading...
From: https://drive.google.com/uc?id=1rd2Tc6oUWBo7JouwexW3ksQ0PaOhUr6h
To: /content/data/code2nl/Cleaned_CodeSearchNet.zip
100% 381M/381M [00:01<00:00, 191MB/s]


In [ ]:
#!unzip Cleaned_CodeSearchNet.zip

In [9]:
#rm Cleaned_CodeSearchNet.zip

In [10]:
#cd ../..

/content


In [5]:
!git clone https://github.com/microsoft/CodeBERT.git

Cloning into 'CodeBERT'...
remote: Enumerating objects: 583, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 583 (delta 45), reused 92 (delta 42), pack-reused 486
Receiving objects: 100% (583/583), 49.56 MiB | 60.20 MiB/s, done.
Resolving deltas: 100% (284/284), done.


In [ ]:
cd /content/CodeBERT/GraphCodeBERT/codesearch

In [ ]:
!unzip dataset.zip

In [8]:
cd /content/CodeBERT/GraphCodeBERT/codesearch/dataset

/content/CodeBERT/GraphCodeBERT/codesearch/dataset


In [ ]:
#cd /content/CodeBERT/GraphCodeBERT/codesearch/dataset/java

In [ ]:
!bash run.sh 

In [13]:
cd /content/CodeBERT/GraphCodeBERT/codesearch/parser

/content/CodeBERT/GraphCodeBERT/codesearch/parser


In [34]:
!bash build.sh

fatal: destination path 'tree-sitter-go' already exists and is not an empty directory.
fatal: destination path 'tree-sitter-javascript' already exists and is not an empty directory.
fatal: destination path 'tree-sitter-python' already exists and is not an empty directory.
fatal: destination path 'tree-sitter-ruby' already exists and is not an empty directory.
fatal: destination path 'tree-sitter-php' already exists and is not an empty directory.
fatal: destination path 'tree-sitter-java' already exists and is not an empty directory.
fatal: destination path 'tree-sitter-c-sharp' already exists and is not an empty directory.


In [14]:
mkdir -p ./saved_models/java

In [15]:
cd /content/CodeBERT/GraphCodeBERT/codesearch/

/content/CodeBERT/GraphCodeBERT/codesearch


In [ ]:
!python run.py \
    --output_dir=parser/saved_models/java \
    --config_name=microsoft/graphcodebert-base \
    --model_name_or_path=microsoft/graphcodebert-base \
    --tokenizer_name=microsoft/graphcodebert-base \
    --lang=java \
    --do_train \
    --train_data_file=dataset/java/train.jsonl \
    --eval_data_file=dataset/java/valid.jsonl \
    --test_data_file=dataset/java/test.jsonl \
    --codebase_file=dataset/java/codebase.jsonl \
    --num_train_epochs 10 \
    --code_length 256 \
    --data_flow_length 64 \
    --nl_length 128 \
    --train_batch_size 32 \
    --eval_batch_size 64 \
    --learning_rate 2e-5 \
    --seed 123456 2>&1| tee parser/saved_models/java/train.log

2022-11-01 04:21:14.658083: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]
Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.

In [ ]:
!python /content/CodeBERT/CodeBERT/code2nl/run.py --do_test --model_type roberta --model_name_or_path microsoft/codebert-base --load_model_path data/model/java/checkpoint-best-bleu/pytorch_model.bin --dev_filename /content/data/code2nl/CodeSearchNet/java/valid.jsonl --test_filename /content/data/code2nl/CodeSearchNet/java/test.jsonl --output_dir model/java --max_source_length 256 --max_target_length 128 --beam_size 10 --eval_batch_size 1000